In [ ]:
%run StoreTweetsBackend.ipynb

In [2]:
# def removePunc(text):
#     return [char for char in text if char not in punctuations]
q={ "dateRetrieved": { "$gt": datetime(1000,1,30),"$lt": datetime(2023,1,30) },'tweetText':{ "$exists": True } }
# { "tweetText":{"$regex" : ".*COPY AND .*"}  }
# { birth: { $gt: new Date('1920-01-01') },death: { $exists: false }}
tweets=getQueryFromMongoDB("Unprocessed",query=q,fields={"_id": 1, "tweetText": 1 }).sort_values('_id')
sentiment140Value=getQueryFromMongoDB("sentiment140").sort_values('_id')
scoretext=sentiment140Value[['SentimentScore','tweetText']].sort_values('SentimentScore')


In [3]:
# print(tweets.size,sentiment140Value.size)
# scoretext.groupby('SentimentScore').describe()
# scoretext.groupby(['length','SentimentScore']).describe()
# scoretext.groupby('length').describe()
#  scoretext[scoretext['length'] == 150]
# sentiment140Value
# tweets
# scoretext['length',"SentimentScore"].plot(bins=20, kind='hist') 
# plt.plot(scoretext['length'],scoretext["SentimentScore"])
# scoretext.hist(column='length', by='SentimentScore', bins=50,figsize=(12,4))

In [4]:
scoretext['hasPunc'] = scoretext['tweetText'].apply(countPunctuations)
# scoretext['hasPunc'] = scoretext['tweetText'].apply(removePunc)
# scoretext.groupby('hasPunc').describe()
#punctuation seems mean messages are going to be longer and more neutral when compared to no punctionation which are most likely negative sentiment

In [5]:
# bow_transformer = CountVectorizer(analyzer=preprocess_String).fit(scoretext['tweetText'])
# bow_transformer.vocabulary_
# bow4 = bow_transformer.transform(scoretext['tweetText'])
# print("Vocab: ",len(bow_transformer.vocabulary_),"\tBOW Shape: ",bow4.shape)
# print('Shape of Sparse Matrix: ', bow4.shape,'\namount of Non-Zero occurences: ', bow4.nnz)
# tfidf_transformer = TfidfTransformer().fit(bow4)
# tfidf4 = tfidf_transformer.transform(bow4)
# print(tfidf4)
# print(tfidf_transformer.idf_[bow_transformer.vocabulary_['u']])
# print(tfidf_transformer.idf_[bow_transformer.vocabulary_['university']])
# messages_tfidf = tfidf_transformer.transform(messages_bow)
# print(messages_tfidf.shape)
# sentimentModel = MultinomialNB().fit(messages_tfidf,scoretext ['SentimentScore'])

In [40]:
ssss=scoretext.sample(frac=1)
msg_train, msg_test, label_train, label_test =train_test_split(ssss['tweetText'], ssss['SentimentScore'],shuffle=True, test_size=0.2)

from sklearn.metrics import confusion_matrix, classification_report, roc_curve
def plot_ROC_NN(y_actual, y_predicted):# one vs rest roc curve
    plt.title("ROC curve")
    y_actual[y_actual==4]=1
    y_predicted[y_predicted==4]=1
    plt.plot(roc_curve(y_actual,y_predicted), alpha=0.5)
    plt.axis([0, 1, 0, 1]) 
    plt.legend(loc="lower right")
    plt.xlabel("False Positive")
    plt.ylabel("True Positive")
def plot_Matrix_NN(y_actual, y_predicted):#plots a matrix
    plt.title("Confusion Matrix")
    matrix=confusion_matrix(y_actual,y_predicted)
    sns.heatmap(matrix, annot=True, fmt='d',xticklabels=range(2),yticklabels=range(2))
    plt.xlabel('Actual')
    plt.ylabel('Predicted')

# predictions = pipeline.predict(msg_test)


from sklearn.svm import SVC
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=preprocess_String)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
#     ('classifier', SVC(gamma='scale')),  # train on TF-IDF vectors w/ Naive Bayes classifier
    ('classifier',  MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])
start=time.time()
pipeline.fit(msg_train,label_train)
print(time.time()-start)

# plot_ROC_NN(label_test,predictions)
# plt.show()
# plot_Matrix_NN(label_test,predictions)
# print(classification_report(label_test,predictions))

14.605939626693726


In [ ]:
# feature_names=BOW.get_feature_names()
# print("Number of features: {}".format(len(feature_names)))
# print("First 20 features:\n{}".format(feature_names[:20]))
# print("Features 20010 to 20030:\n{}".format(feature_names[20010:20030]))
# print("Every 2000th feature:\n{}".format(feature_names[::2000]))

In [ ]:
scores = cross_val_score(LogisticRegression(), msg_train,label_train,cv=2)
# print("Mean cross-validation accuracy: {:.2f}".format(np.mean(scores)))

In [ ]:
print(scores)

In [ ]:
vect = CountVectorizer(max_features=10000, max_df=.15)
X = vect.fit_transform(msg_train)

In [ ]:
vect = CountVectorizer()
vect = CountVectorizer().fit(msg_train)
X_train = vect.transform(msg_train)
print("X_train:\n{}".format(repr(msg_train)))

In [ ]:
scores = cross_val_score(LogisticRegression(), X_train, label_train, cv=5)

In [ ]:
print("Mean cross-validation accuracy: {:.2f}".format(np.mean(scores)))

In [ ]:
print(scores)

In [ ]:

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)

In [ ]:
grid.fit(X_train, label_train)

In [ ]:
print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Best parameters: ", grid.best_params_)
X_test = vect.transform(msg_test)
print("Test score: {:.2f}".format(grid.score(X_test, label_test)))

In [ ]:
vect = CountVectorizer(min_df=5).fit(text_train)

In [ ]:
vect = CountVectorizer(max_features=10000, max_df=.1)
X = vect.fit_transform(msg_train)

In [ ]:
lda = LatentDirichletAllocation(n_components=10, learning_method="batch",max_iter=2, random_state=0)
document_topics = lda.fit_transform(X[:100])

In [ ]:
print("lda.components_.shape: {}".format(lda.components_.shape))

In [ ]:
 # for each topic (a row in the components_), sort the features (ascending).
# Invert rows with [:, ::-1] to make sorting descending
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
# get the feature names from the vectorizer:
feature_names = np.array(vect.get_feature_names())

In [ ]:
# Print out the 10 topics:

mglearn.tools.print_topics(topics=range(10), feature_names=feature_names,
                           sorting=sorting, topics_per_chunk=5, n_words=10)

In [ ]:
lda100 = LatentDirichletAllocation(n_components=100, learning_method="batch",
                                   max_iter=25, random_state=0)
document_topics100 = lda100.fit_transform(X[:1000])

In [ ]:
topics = np.array([7, 16, 24, 25, 28, 36, 37, 41, 45, 51, 53, 54, 63, 89, 97])

In [ ]:
sorting = np.argsort(lda100.components_, axis=1)[:, ::-1]
feature_names = np.array(vect.get_feature_names())
mglearn.tools.print_topics(topics=topics, feature_names=feature_names,
                           sorting=sorting, topics_per_chunk=5, n_words=20)

In [ ]:
# sort by weight of "music" topic 45
music = np.argsort(document_topics100[:, 45])[::-1]
# print the five documents where the topic is most important
for i in music[:10]:
      # show first two sentences
    
    break
music[:10]
X[620]

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 10))
topic_names = ["{:>2} ".format(i) + " ".join(words)
               for i, words in enumerate(feature_names[sorting[:, :2]])]
# two column bar chart:
for col in [0, 1]:
    start = col * 50
    end = (col + 1) * 50
    ax[col].barh(np.arange(50), np.sum(document_topics100, axis=0)[start:end])
    ax[col].set_yticks(np.arange(50))
    ax[col].set_yticklabels(topic_names[start:end], ha="left", va="top")
    ax[col].invert_yaxis()
    ax[col].set_xlim(0, 2000)
    yax = ax[col].get_yaxis()
    yax.set_tick_params(pad=130)
plt.tight_layout()